# Database preparation

#### 1. Documentation
- <https://praw.readthedocs.io/en/latest/>

#### 2. Examples
- <https://praw.readthedocs.io/en/latest/tutorials/comments.html>
- <https://www.sciencedirect.com/topics/computer-science/centrality-measure>
- <https://aksakalli.github.io/2017/07/17/network-centrality-measures-and-their-visualization.html>
- <https://cambridge-intelligence.com/keylines-faqs-social-network-analysis/>
- <https://medium.com/analytics-vidhya/simplifying-social-media-sentiment-analysis-using-vader-in-python-f9e6ec6fc52f>

#### Libraries

In [1]:
#Data trancformation
import sqlite3
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

#Reddit extraction
import praw

#Time 
import time

# graph packages
import networkx as nx
import igraph
from igraph import Graph

# NLP
from nltk.sentiment.vader import SentimentIntensityAnalyzer as SIA
import re

In [2]:
import datetime as dt
from datetime import datetime

#### Connect to reddit

In [3]:
reddit = praw.Reddit(client_id='pG_dvbbdt151Rw', \
                     client_secret='KNp35NWOZ7FvjyF5haiPUAKSd8o', \
                     user_agent='O_Auth_app', \
                     username='_Alex_shadow_', \
                     password='2424678Rise')

#### Extraction of data

In [4]:
def data_extraction(database_list):
    
    conn = sqlite3.connect('{0}.db'.format(database_list))
    
    # 1) Preparing a list of post ids
    list_of_posts = pd.read_sql('''SELECT distinct post_id
                                    FROM Reddit
                                    GROUP BY post_id
                                    ORDER BY  max(score) desc
                                    LIMIT 110''', conn).values.tolist()
    
    # 2) Empty dictionary
    topics_dict = {"author": [],
                   "body":[],
                   "created_utc":[],
                   "id": [],
                   "post_id": [],
                   "parent_id": [],
                   "score": [],
                   "subreddit_id": [],
                   "num_comments": []               
                  }
    
    # 3) Extracting posts
    for elements in list_of_posts:
        submission  = reddit.submission(id=str(elements).strip('[').strip(']').strip("'"))

        topics_dict["author"].append(submission.author)
        topics_dict["body"].append(submission.title+" "+submission.selftext)
        topics_dict["created_utc"].append(submission.created_utc)
        topics_dict["id"].append(submission.id)
        topics_dict["post_id"].append(submission.name)
        topics_dict["parent_id"].append(submission.name)
        topics_dict["score"].append(submission.score)
        topics_dict["subreddit_id"].append(submission.subreddit_id)
        topics_dict["num_comments"].append(submission.num_comments)
        
    df_post=pd.DataFrame.from_dict(topics_dict, orient='index').transpose()
    df_post['type']="Post"
    df_post["subreddit_id"]=df_post["subreddit_id"].str[3:]    
    df_post["parent_id"]=df_post["parent_id"].str[3:]
    df_post["post_id"]=df_post["post_id"].str[3:]
    
    # 4) Extracting comments
    df_comments = pd.read_sql('''SELECT
                                    author,
                                    body,
                                    created_utc,
                                    id,
                                    post_id,
                                    parent_id,
                                    score,
                                    subreddit_id

                                 FROM Reddit
                                 WHERE post_id in (SELECT 
                                                        distinct post_id
                                                    FROM Reddit
                                                    GROUP BY post_id
                                                    ORDER BY  max(score) desc 
                                                    LIMIT 110)
                                 ''', conn)

    df_comments['type']="Comments"    
    
    # 5) Collect all together
    df_all = pd.concat([df_comments, df_post], join='outer', sort=False).replace({np.nan: None}).reset_index(drop=True)

    return df_all


#### Preparation of statistics

In [5]:
def data_preparation(df):

    # list of posts
    list_of_posts=list(pd.unique(df["post_id"].values))
    
    # Empty dataframe
    df_merge = pd.DataFrame()
    
    # cycling through posts 
    for elements in list_of_posts:
        
        # filter dataset 
        df_filter=df.loc[df['post_id'] == elements].copy().sort_values(by=['id'], ascending=False)

        G = nx.from_pandas_edgelist(df=df_filter, source="id", target="parent_id", create_using=nx.Graph()) 
        if nx.is_connected(G)==False:
            print("Dropped: {}".format(elements))
            continue 
            
            
        # 1. Calculate level depth
        depth_levels_from_root = pd.DataFrame(list(nx.single_source_shortest_path_length(G, source=elements).items()), columns=['id', 'level']).sort_values(by=['id'], ascending=False)
        
        # 2. Calculate number of connected nodes
        n_connections = pd.DataFrame(list( dict(G.degree()).items() ), columns=['id', 'n_conn']).sort_values(by=['id'], ascending=False)
        
        # 2. Add number of comments and month
        df_filter["num_comments"]=df_filter.loc[df_filter['type'] == 'Post', "num_comments"].values[0]
        
        # 3. finding how many seconds passed for post
        def get_yyyy_mm_dd_from_utc(dt):
            date = datetime.utcfromtimestamp(dt) 
            return date 
        df_filter['date'] = df_filter['created_utc'].astype('int').apply(lambda d: get_yyyy_mm_dd_from_utc(d))   

        # 4. Adding month and quarter labels
        df_filter['year'] = df_filter['date'].dt.year 
        df_filter['month'] = df_filter['date'].dt.month 
        df_filter['quarter'] = df_filter['date'].dt.quarter
        
        # 5. Grouping by time
        t_min = min(df_filter["date"])
        t_max = max(df_filter["date"])
        diff=t_max-t_min
        t_diff=diff.days*24
        time_int=(t_diff,730,168,24,16,8)
        label=("<=8 hours","<=16 hours","<=1 day","<=7 days","<=1 month","more than 1 month")
        num=5
        for elements in time_int:
            t = t_min + dt.timedelta(hours=elements) 
            df_filter.loc[df_filter["date"] <= t, 'time_group'] = num 
            df_filter.loc[df_filter["date"] <= t, 'time_label'] = label[num] 
            num-=1
        df_filter['time_group']=df_filter['time_group'].astype('int8')
                
        # 6. Adding info about previous author               
        df_user=df_filter[["id","author","date"]]
        df_user = df_user.rename(columns={'author': 'parent_author',
                                        'id': 'parent_id',
                                        'date': 'parent_date'})
        df_filter=pd.merge(df_filter, df_user, on='parent_id')
        
        df_filter['age_comment'] = (df_filter.date - df_filter.parent_date).dt.total_seconds().astype('str')
        df_filter['age_post'] = (df_filter.date - min(df_filter.date)).dt.total_seconds().astype('str')
               
        # 7. Count number of text in each comment
        df_filter['body_length']=df_filter['body'].str.len()
        
        # append data
        df_filter=pd.merge(df_filter, depth_levels_from_root, on='id')       
        df_filter=pd.merge(df_filter, n_connections, on='id')
        df_merge=df_merge.append(df_filter, sort=True)
        
    return df_merge    

#### Calculation of centrality measures

In [6]:
def centrality_measures(df):

    # list of posts
    list_of_posts=list(pd.unique(df["post_id"].values))
    
    # Empty dataframe
    df_merge = pd.DataFrame()

    # cycling through posts 
    for elements in list_of_posts:
      
        # filter dataset 
        df_filter=df.loc[df['post_id'] == elements].copy().sort_values(by=['id'], ascending=False)
        
        # Directed graph
        G = nx.from_pandas_edgelist(df=df_filter, source="id", target="parent_id", create_using=nx.DiGraph()) 
        
        # 1. Calculate page rank
        pr = pd.DataFrame(list(nx.pagerank(G, alpha=0.8).items()), columns=['id', 'centrality_page_rank']).sort_values(by=['id'], ascending=False)  
        
        # 2. Calculate eigenvector
        eigenvector = pd.DataFrame(list(nx.eigenvector_centrality_numpy(G).items()), columns=['id', 'centrality_eigenvector']).sort_values(by=['id'], ascending=False)  
        
        # 3. Calculate degree centrality
        degree = pd.DataFrame(list(nx.degree_centrality(G).items()), columns=['id', 'centrality_degree']).sort_values(by=['id'], ascending=False)  
        
        # 4. Calculate closeness centrality
        closeness = pd.DataFrame(list(nx.closeness_centrality(G).items()), columns=['id', 'centrality_closeness']).sort_values(by=['id'], ascending=False)  
         
        # Directed graph
        g = Graph()
        tuples = [tuple(x) for x in df_filter[["id", "parent_id"]].values]
        G = igraph.Graph.TupleList(tuples, directed = True)
        
        # 5. Calculate betweeness centrality
        df_filter["centrality_betweeness"]= [float(i)/max(G.betweenness()) for i in G.betweenness()]

        # Merge all together
        df_filter=pd.merge(df_filter, pr, on='id')          
        df_filter=pd.merge(df_filter, eigenvector, on='id')    
        df_filter=pd.merge(df_filter, degree, on='id')    
        df_filter=pd.merge(df_filter, closeness, on='id')   
        
        # append data
        df_merge=df_merge.append(df_filter, sort=True)
    
    return df_merge    

#### NLP analysis

In [7]:
def decontracted(phrase):
    # removing hyperlinks
    phrase = re.sub(r'\b(?:(?:https?|ftp)://)?\w[\w-]*(?:\.[\w-]+)+\S*', ' ', phrase)
    phrase = re.sub(r"http:", "", phrase)
    
    # dropping unwanted symbols:
    phrase = re.sub(r"&lt;", "", phrase)
    phrase = re.sub(r"&gt;", "", phrase)
    phrase = re.sub(r"&le;", "", phrase)
    phrase = re.sub(r"&ge;", "", phrase)

    # specific
    phrase = re.sub(r"won't", "will not", phrase)
    phrase = re.sub(r"can\'t", "can not", phrase)

    # general
    phrase = re.sub(r"n\'t", " not", phrase)
    phrase = re.sub(r"\'re", " are", phrase)
    phrase = re.sub(r"\'s", " is", phrase)
    phrase = re.sub(r"\'d", " would", phrase)
    phrase = re.sub(r"\'ll", " will", phrase)
    phrase = re.sub(r"\'t", " not", phrase)
    phrase = re.sub(r"\'ve", " have", phrase)
    phrase = re.sub(r"\'m", " am", phrase)
    
    
    return phrase

In [8]:
def nlp_scoring(df):
    sia = SIA()
    results = []

    for line in df['body']:
        pol_score = sia.polarity_scores(line)
        results.append(pol_score)
        
    df_text = pd.DataFrame.from_records(results)
    
    #Assign label and nlp score
    df['nlp_score'] = df_text['compound']
   

    df['nlp_group'] = 4
    df['nlp_label'] = "Very Positive" 

    df.loc[df["nlp_score"] <= 0.5, 'nlp_group'] = 3
    df.loc[df["nlp_score"] <= 0.5, 'nlp_label'] = "Positive" 

    df.loc[df["nlp_score"] <= 0.05, 'nlp_group'] = 2
    df.loc[df["nlp_score"] <= 0.05, 'nlp_label'] = "Neutral" 

    df.loc[df["nlp_score"] <= -0.05, 'nlp_group'] = 1
    df.loc[df["nlp_score"] <= -0.05, 'nlp_label'] = "Negative" 

    df.loc[df["nlp_score"] <= -0.5, 'nlp_group'] = 0 
    df.loc[df["nlp_score"] <= -0.5, 'nlp_label'] = "Very Negative" 
    
    return df

#### Formatting

In [9]:
def data_formatting(df):

    # post info
    df['author']=df['author'].astype('str')  
    df['body']=df['body'].astype('str')
    df['id']=df['id'].astype('str')
    df['post_id']=df['post_id'].astype('str')
    df['subreddit_id']=df['subreddit_id'].astype('str')
    
    df['score']=df['score'].astype('int16')
    df['type']=df['type'].astype('str')
    df['level']=df['level'].astype('int16')
    df['n_conn']=df['n_conn'].astype('int16')
    df['num_comments']=df['num_comments'].astype('int16')
    df['body_length']=df['body_length'].astype('int16')
    
    # centrality measures
    df['centrality_betweeness']=df['centrality_betweeness'].round(6).astype('float16')
    df['centrality_closeness']=df['centrality_closeness'].round(6).astype('float16')
    df['centrality_degree']=df['centrality_degree'].round(6).astype('float16')  
    df['centrality_eigenvector']=df['centrality_eigenvector'].round(6).astype('float16')
    df['centrality_page_rank']=df['centrality_page_rank'].round(6).astype('float16')
    
    #time 
    df['created_utc']=df['created_utc'].astype('int16')
    df['date']=df['date'].astype('str')
    
    df['month']=df['month'].astype('int16')
    df['year']=df['year'].astype('int16')
    df['quarter']=df['quarter'].astype('int16')  
    
    df['age_comment']=df['age_comment'].astype('str')
    df['age_post']=df['age_post'].astype('str')
        
    df['time_group']=df['time_group'].astype('int16')    
    df['time_label']=df['time_label'].astype('str')  
    
    #previous post
    df['parent_author']=df['parent_author'].astype('str')
    df['parent_id']=df['parent_id'].astype('str')
    df['parent_date']=df['parent_date'].astype('str')    

    #NLP
    df['nlp_score']=df['nlp_score'].round(6).astype('float16')
    df['nlp_group']=df['nlp_group'].astype('int8')
    df['nlp_label']=df['nlp_label'].astype('str')
    
    return df

#### Main code

In [10]:
def data_prep(data_list, out_database):
    
    # Empty dataframe
    df_merge = pd.DataFrame()

    start_time = time.time()   
    print("---------------------------------------------------")
    print("Start time: {}".format(start_time))
    conn = sqlite3.connect(out_database)
    for elements in data_list:
        print("---------------------------------------------------")
        print("Database : {}".format(elements))       
        step0 = time.time()
        # 1) Exracting comments
        df_1 = data_extraction(elements)
        step1 = time.time()

        print("Step 1 done in: {}".format(step1-step0))
        
        # 2) Preparing data
        df_2 = data_preparation(df_1)
        step2 = time.time()
        print("Step 2 done in: {}".format(step2-step1))
        
        # 3) centrality calculation
        df_3 = centrality_measures(df_2)
        step3 = time.time()  
        print("Step 3 done in: {}".format(step3-step2))
        
        # 4) NLP analysis
        df_4 = nlp_scoring(df_3)
        step4 = time.time()
        print("Step 4 done in: {}".format(step4-step3))
        
        # 5) data format
        df_5 = data_formatting(df_4)
        step5 = time.time()
        print("Step 5 done in: {}".format(step5-step4))
        
        df_5.to_sql('Reddit', conn, if_exists='append', index=False)
        #df_5.to_sql('Reddit', conn, if_exists='replace', index=False)
        
    print("Total time: {}".format(time.time()-start_time))  

#### List of datasets to process

In [11]:
data_list=["Data/initial/2016/10_2016",
           "Data/initial/2016/11_2016",
           "Data/initial/2016/12_2016"]
data_prep(data_list, out_database="Data/trump_election.db")

---------------------------------------------------
Start time: 1594742899.9225092
---------------------------------------------------
Database : Data/initial/2016/10_2016
Step 1 done in: 649.4753940105438
Step 2 done in: 46.22297739982605
Step 3 done in: 1853.0605521202087
Step 4 done in: 198.1242334842682
Step 5 done in: 2.5012917518615723
---------------------------------------------------
Database : Data/initial/2016/11_2016
Step 1 done in: 779.5968918800354
Step 2 done in: 64.56959295272827
Step 3 done in: 2177.4735159873962
Step 4 done in: 264.6592037677765
Step 5 done in: 3.468505620956421
---------------------------------------------------
Database : Data/initial/2016/12_2016
Step 1 done in: 731.5279066562653
Step 2 done in: 55.55265522003174
Step 3 done in: 2003.5390720367432
Step 4 done in: 261.5582118034363
Step 5 done in: 3.5097899436950684
Total time: 9113.05395936966


In [12]:
data_list=["Data/initial/2012/01_2012",
           "Data/initial/2012/02_2012",
           "Data/initial/2012/03_2012",
           "Data/initial/2012/04_2012",
           "Data/initial/2012/05_2012",
           "Data/initial/2012/06_2012",
           "Data/initial/2012/07_2012",
           "Data/initial/2012/08_2012",
           "Data/initial/2012/09_2012",
           "Data/initial/2012/10_2012",
           "Data/initial/2012/11_2012",
           "Data/initial/2012/12_2012"]
data_prep(data_list, out_database="Data/obama_election.db")

---------------------------------------------------
Start time: 1594752013.378426
---------------------------------------------------
Database : Data/initial/2012/01_2012
Step 1 done in: 648.3846743106842
Dropped: nxu96
Dropped: o0dm1
Dropped: o3cc7
Step 2 done in: 18.137250661849976
Step 3 done in: 183.8024160861969
Step 4 done in: 79.60364484786987
Step 5 done in: 1.005279541015625
---------------------------------------------------
Database : Data/initial/2012/02_2012
Step 1 done in: 675.2601225376129
Dropped: p8i1n
Dropped: p9m43
Dropped: pa3yr
Dropped: pb9nb
Dropped: pbu0z
Dropped: pbv6n
Dropped: pc2nm
Dropped: pcato
Dropped: pd9rv
Dropped: pdbrs
Dropped: pdc1y
Dropped: pddgr
Dropped: pexk3
Step 2 done in: 15.499390602111816
Step 3 done in: 144.71490097045898
Step 4 done in: 75.06330585479736
Step 5 done in: 0.8365685939788818
---------------------------------------------------
Database : Data/initial/2012/03_2012
Step 1 done in: 652.6654098033905
Dropped: qwgca
Dropped: qxo6u
Ste

In [17]:
#conn = sqlite3.connect("test.db")
conn = sqlite3.connect("Data/trump_election.db")
df_test = pd.read_sql('''SELECT * FROM Reddit limit 10''', conn)
df_test

,age_comment,age_post,author,body,body_length,centrality_betweeness,centrality_closeness,centrality_degree,centrality_eigenvector,centrality_page_rank,...,quarter,score,subreddit_id,time_group,time_label,type,year,nlp_score,nlp_group,nlp_label
0,2163819.0,2206052.0,Aegisflame,Actually then we're almost on the same page. M...,145,0.000000,0.000000,0.000937,0.0,0.000187,...,1,1,2cneq,4,<=1 month,Comments,2016,-0.612305,3,Negative
1,11032.0,326269.0,LucidicShadow,"Interesting read. A little anecdotal, but inte...",69,0.026505,0.000000,0.000937,0.0,0.000187,...,1,1,2cneq,3,<=7 days,Comments,2016,0.612793,1,Positive
2,34897.0,315237.0,cuntipede,It can return a positive result when you smoke...,183,0.000000,0.000937,0.001874,0.0,0.000337,...,1,1,2cneq,3,<=7 days,Comments,2016,0.557617,1,Positive
3,252185.00000000003,303169.0,tacosaladchupacabra,&gt;Checkpoints are constitutional because a l...,152,0.019272,0.000000,0.000937,0.0,0.000187,...,1,1,2cneq,3,<=7 days,Comments,2016,-0.051605,2,Neutral
4,819.0,280340.0,LucidicShadow,"Ok, decided to look this shit up.\n\nAccording...",1188,0.033722,0.001250,0.001874,0.0,0.000457,...,1,1,2cneq,3,<=7 days,Comments,2016,-0.891602,3,Negative
5,239747.00000000003,279521.0,cuntipede,Ticket you? LOL. It is a full-blown drug drivi...,108,0.000000,0.001406,0.001874,0.0,0.000553,...,1,1,2cneq,3,<=7 days,Comments,2016,-0.352539,2,Neutral
6,42012.0,272982.0,cjlm,"If you refuse a breath test here, you get auto...",232,0.016861,0.000000,0.000937,0.0,0.000187,...,1,1,2cneq,3,<=7 days,Comments,2016,-0.202271,2,Neutral
7,186140.0,231073.0,rivermonkey66,We have a group called MADD which lied to and ...,136,0.043365,0.000000,0.000937,0.0,0.000187,...,1,0,2cneq,3,<=7 days,Comments,2016,-0.659668,3,Negative
8,185682.0,230970.0,rivermonkey66,"People refuse, the evidence is not clear, and ...",138,0.048187,0.000937,0.001874,0.0,0.000337,...,1,0,2cneq,3,<=7 days,Comments,2016,-0.200073,2,Neutral
9,174026.0,230349.0,rivermonkey66,"Missing the point. *This should not exist, 20...",61,0.000000,0.000000,0.000937,0.0,0.000187,...,1,1,2cneq,3,<=7 days,Comments,2016,-0.295898,2,Neutral


In [18]:
#conn = sqlite3.connect("test.db")
conn = sqlite3.connect("Data/obama_election.db")
df_test = pd.read_sql('''SELECT * FROM Reddit limit 10''', conn)
df_test

,age_comment,age_post,author,body,body_length,centrality_betweeness,centrality_closeness,centrality_degree,centrality_eigenvector,centrality_page_rank,...,quarter,score,subreddit_id,time_group,time_label,type,year,nlp_score,nlp_group,nlp_label
0,0.0,0.0,TruthToPower1,"Congress is at 5% approval, and 86% say Congre...",101,0.000000,0.319092,0.292480,1.0,0.551270,...,1,2510,2cneq,0,<=8 hours,Post,2012,0.726562,1,Positive
1,29579.000000000004,1437407.0,wharpudding,"Hey, you're right. We could have gotten rid o...",132,0.000000,0.000000,0.001493,0.0,0.000298,...,1,0,2cneq,4,<=1 month,Comments,2012,0.763184,1,Positive
2,29409.000000000004,1437237.0,[deleted],[deleted],9,0.020828,0.000000,0.001493,0.0,0.000298,...,1,1,2cneq,4,<=1 month,Comments,2012,0.000000,2,Neutral
3,1365677.0,1407916.0,sixpackistan,"There are no term limits and, no, they're not ...",269,0.000000,0.000000,0.001493,0.0,0.000298,...,1,1,2cneq,4,<=1 month,Comments,2012,-0.526855,3,Negative
4,1371077.0,1407828.0,sixpackistan,"Term limits != one term. It means, you can't ...",162,0.000000,0.002985,0.004478,0.0,0.000775,...,1,0,2cneq,4,<=1 month,Comments,2012,0.000000,2,Neutral
5,50262.0,429393.0,[deleted],[deleted],9,0.006943,0.000000,0.001493,0.0,0.000298,...,1,1,2cneq,3,<=7 days,Comments,2012,0.000000,2,Neutral
6,366334.0,379131.0,F_IT,Who voted the fuckers in? Thats right.... Amer...,92,0.027771,0.001493,0.002985,0.0,0.000537,...,1,1,2cneq,3,<=7 days,Comments,2012,-0.636230,3,Negative
7,8583.0,257214.00000000003,Goldenrule-er,"Honestly though, forgive me for being so crass...",372,0.000000,0.000000,0.001493,0.0,0.000298,...,1,1,2cneq,3,<=7 days,Comments,2012,0.691406,1,Positive
8,198059.0,248631.00000000003,[deleted],It could also mean that you're a spoiled child.,47,0.006943,0.001493,0.002985,0.0,0.000537,...,1,1,2cneq,3,<=7 days,Comments,2012,0.000000,2,Neutral
9,105719.0,154869.0,MdxBhmt,And it's also sad that some of those good peop...,155,0.131958,0.000000,0.001493,0.0,0.000298,...,1,2,2cneq,3,<=7 days,Comments,2012,-0.817383,3,Negative


- posts: 2016 - 1318, 2012 - 1283 
- observations: 2016 - 4496239, 2012 - 2044993 